In [1]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
import pandas as pd
from tabulate import tabulate
from cu2o_bulk import c2x2, cu2o_bulk, cu2o100, Oterm1x1, c2x2, slab3011

bulk = cu2o_bulk()
bulk.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
bulk_oxygens = bulk[bulk.symbols=='O']
print(f"Number of Oxygen atoms in bulk: {len(bulk_oxygens)}")
#write('bulk.xyz', bulk)

n_layers = { 3,4,5,6,7,8,9,10}
Cu2Otable={}

n_layers_list=[]
slab_oxygens_list=[]
E_surf_list=[]


for n_layers in range(3,11):
  vacuum=10
  slab = slab3011(bulk, n_layers, vacuum)
  #write('slab.xyz', slab)
  slab_oxygens = slab[slab.symbols=='O']
  Cu2Otable[n_layers]=len(slab_oxygens)
  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")

  slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cpu')
  E_slab = slab.get_potential_energy()
  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_cleav=}')

  qn = LBFGS(slab, trajectory='slab3011.traj')
  qn.run(fmax=0.01)
  E_slab = slab.get_potential_energy()

  ##Calc Surface Energy
  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_surf=}')

  n_layers_list.append(n_layers)
  slab_oxygens_list.append(len(slab_oxygens))
  E_surf_list.append(E_surf)

df = pd.DataFrame({'Number of Layers': n_layers_list, 'Number of Oxygen Atoms':slab_oxygens_list, 'Surface Energy': E_surf_list})
table=tabulate(df, headers = 'keys', tablefmt = 'fancy_grid')
df.to_csv('slab3011.csv', index=False)
print(table)


<>:21: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:21: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


Cached MACE model to /home/lana/.cache/mace/5f5yavf3
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5f5yavf3
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.